In [79]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from scipy.stats import pearsonr
from scipy.sparse import linalg
from scipy.linalg import solve
import matplotlib.pyplot as plt

In [74]:
input_weight = 1.;
spectral_radius = 0.9;
regularization = 1e-4;
average_degree = 20;
leakage = 0;
transient_length = 1000;
train_length = 100000;
approx_num_nodes = 1000;

reader = pd.read_csv('KS_train_input_sequence_N64.csv')
train_input_sequence = reader.to_numpy().transpose()
reader = pd.read_csv('KS_test_input_sequence_N64.csv')
test_input_sequence = reader.to_numpy().transpose()
reader = pd.read_csv('KS_pred_start_indices_N64.csv')
start_iter = reader.to_numpy()

input_size = train_input_sequence.shape[0]
num_nodes = int(np.ceil(approx_num_nodes/input_size)*input_size);

A_mat = np.random.rand(num_nodes,num_nodes)
threshold = (num_nodes**2 - average_degree*num_nodes)/(num_nodes**2)
Ap_mat = np.zeros((num_nodes,num_nodes))
for x in range(num_nodes):
    for y in range(num_nodes):
        if A_mat[x,y]>threshold:
            Ap_mat[x,y] = A_mat[x,y]

Ap_mat = sparse.coo_matrix(Ap_mat)
eg = linalg.eigs(Ap_mat, k = 1, return_eigenvectors=False)
Ap_mat = spectral_radius/np.abs(eg[0])*Ap_mat

q = int(np.floor(num_nodes/(input_size)))
win = np.zeros((num_nodes,input_size))
for i in range(input_size):
    np.random.seed(i)
    ip = (-1 + 2*np.random.randn(q));
    win[i*q:(i+1)*q,i] = input_weight*ip;
    
x = np.zeros(num_nodes)
states = np.zeros((num_nodes, train_length))

In [75]:
for t in range(transient_length):
    x = leakage*x + (1-leakage)*np.tanh(Ap_mat.dot(x) + np.matmul(win,train_input_sequence[:,t]))
    
states[:,0] = x;

for t in range(train_length-1):
    states[:,t+1] = leakage*states[:,t] + (1-leakage)*\
    np.tanh(Ap_mat.dot(states[:,t]) + np.matmul(win,train_input_sequence[:,t+transient_length]))
    
aug_states = np.copy(states)
aug_states[::2,:] = np.power(states[::2,:],2)
idenmat = regularization*sparse.identity(num_nodes)
data_trstates = np.matmul(train_input_sequence[:,transient_length:transient_length+train_length],\
                         np.transpose(aug_states))
states_trstates = np.matmul(aug_states,np.transpose(aug_states))

In [80]:
wout = np.transpose(solve(np.transpose(states_trstates + idenmat),np.transpose(data_trstates)))

In [81]:
test_results = test_reservoir(win,Ap_mat,wout,test_input_sequence,start_iter)
A_new = np.maximum(ones(num_nodes,num_nodes), A_mat + threefactor(wout,aug_states))

[[ 1.02971763e-02 -1.66581605e+00  4.06959559e-02 ...  1.07122558e-02
  -8.34844807e-04  3.15668597e-02]
 [ 7.17322446e-03 -1.57744151e+00  1.45837362e-02 ... -1.97929086e-03
  -1.40837512e-02  4.44019154e-02]
 [ 5.26264294e-03 -1.27164604e+00 -1.00548658e-02 ... -2.28531146e-02
  -2.43469609e-02  3.73643016e-02]
 ...
 [ 7.01955990e-03 -5.87887113e-01  5.19202256e-02 ...  5.50276506e-03
   2.66558216e-02 -5.88076185e-04]
 [ 1.03985751e-02 -1.09631079e+00  6.27956026e-02 ...  8.41994039e-03
   2.29912140e-02  5.65112610e-03]
 [ 1.03740042e-02 -1.47824051e+00  5.74528742e-02 ...  1.42262600e-02
   1.34585784e-02  2.59330866e-02]]
